In [216]:
# Data Sources:
# Wikipedia
# arxiv.org

In [220]:
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.firefox.options import Options

from bs4 import BeautifulSoup
import requests
from random_word import RandomWords

In [270]:
def getWikiText(soupObject): 
    text = ''
    
    for table in soup.find_all('table'):
        table.extract()
    for image in soup.find_all('img'):
        image.extract()
    for list_ in soup.find_all('ul'):
        list_.extract()
    
    for elem in soup.find_all(['p', 'h1', 'h2', 'h3']):
        if elem.has_attr('id'):
            if elem['id'] == 'firstHeading':
                text = text + elem.get_text() + '\n'

        if elem.name == 'h2' or elem.name == 'h3':
            header = elem.find(class_='mw-headline')
            if header is not None:
                headerText = header.get_text()
                if headerText not in ['See also','References','Further reading', 'External links']:
                    text = text + '\n' + header.get_text() + '\n'
                    
        if elem.name == 'p':
            text = text + elem.get_text()
    return text

In [272]:
baseUrl = 'https://en.wikipedia.org/wiki/'
numberOfUrlsToTry = 10
numerofUrlsToDownload = 100
numberOfSuccessfulDownloads = 0
numberOfAttemptDownloads = 0
maxAttemptsAllowed = 2*numerofUrlsToDownload

cwd = Path().cwd()
randomWordGenerator = RandomWords()
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(options=options)



while numberOfSuccessfulDownloads < numerofUrlsToDownload:
    
    if numberOfAttemptDownloads>=maxAttemptsAllowed:
        break
    
    urls = []
    for idx in range(numberOfUrlsToTry):
        randomWord = randomWordGenerator.get_random_word()
        url = baseUrl + randomWord
        urls.append(url)

    for url in urls:
        response = requests.get(url)
        html_doc = response.text
        encoding = response.encoding
        numberOfAttemptDownloads +=1
        
        if 'Wikipedia does not have an article with this exact name.' not in html_doc:
            folderName = url.split('/')[-1]
            folderPath = cwd.joinpath('inputData')
            folderPath = folderPath.joinpath(folderName)

            if not folderPath.exists():
                folderPath.mkdir()

            screenshotPath = folderPath.joinpath('screenshot.png')
            textPath = folderPath.joinpath('text.txt')

            soup = BeautifulSoup(html_doc, 'html.parser')
            text = getWikiText(soup)
            with textPath.open('w', encoding=encoding) as f:
                f.write(text)

            driver.get(url)
            driver.save_full_page_screenshot(str(screenshotPath))

            numberOfSuccessfulDownloads+=1
            
driver.quit()